# flavio tutorial

## Part 3: Wilson coefficients

### Assumptions about new physics (NP) in flavio

- NP enters observables via local interactions (operators) among known particles
- NP only affects *predictions*, not *measurements*

### Operators & Wilson coefficients

- The *Wilson coefficients* are the couplings strengths of the local operators
- NP is dealt with in flavio by providing the numerical values of these Wilson coefficients
- In flavour physics we usually deal with Wilson coefficients in the weak effective theory (WET) below the electroweak scale

### Hierarchy of effective field theories (EFTs)

![](img/EFT-hierarchy-WCxf.svg)

## Wilson coefficient exchange format (WCxf)

Before discussing how to specify Wilson coefficients in flavio, let's have look at a **new** data format defined in this write-up:

> "WCxf: an exchange format for Wilson coefficients beyond the Standard Model"  
> J. Aebischer et al.  
> [arXiv:1712.05298](https://arxiv.org/abs/1712.05298)

<https://wcxf.github.io>

### WCxf: motivation

- WCxf allows to unambiguously define EFTs and bases of Wilson coefficients by depositing YAML definition files in a public repository
- This allows codes like flavio to work with different bases, conventions, and even EFTs
- flavio defines its own WCxf basis for WET, WET-4, and WET-3

See <https://wcxf.github.io/bases.html>

### Basis facts about WCs in WCxf and flavio

- Wilson coefficients are always numbers (not matrices)
- can be real or complex depending on operator
- Wilson coefficients refer to **NP contributions only**, i.e. vanish in the SM

Example: `C9_bsmumu = -1` means that $C_9=C_9^\text{SM} - 1$

- Dimensionful coefficients are in appropriate powers of GeV

## `WilsonCoefficients` class

In [ ]:
import flavio
wc = flavio.WilsonCoefficients()
wc.set_initial({'C9_bsmumu': -1, 'CVLR_bsbs': 1e-6}, scale=160)

In [ ]:
wc

In [ ]:
wc.get_initial_wcxf

In [ ]:
wc.get_wc(sector='sbsb', scale=4.2, par=None)

### Comment on the renormalization group running

The RG evolution was completely rewritten in flavio v0.27 and is now based on the standalone package [`wetrunner`](https://github.com/DavidMStraub/python-wetrunner) which is based on [1704.06639](https://arxiv.org/abs/1704.06639) (see also [1704.04504](https://arxiv.org/abs/1704.04504))

## Using SMEFT

The WCxf Python package allows to automatically match SMEFT Wilson coefficients to the WET in the flavio basis.

Example: $O_{qq}^{(1),  2323} = \left( \bar q_2 \gamma_\mu q_3 \right) \left( \bar q_2 \gamma^\mu q_3 \right)$

In [ ]:
import wcxf
wcxf_smeft = wcxf.WC(eft='SMEFT', basis='Warsaw', scale=160,
                   values={'qq1_2323': 1e-6})
wcxf_flavio = wcxf_smeft.match('WET', 'flavio')
wc_flavio = flavio.WilsonCoefficients()
wc_flavio.set_initial_wcxf(wcxf_flavio)

In [ ]:
wcxf_smeft

In [ ]:
wc_flavio

## NP predictions

Having defined a Wilson Coefficient instance, predictions can be computed in the presence of NP.

Example: $R_K=\text{BR}(B\to K\mu\mu)/\text{BR}(B\to Kee)$

In [ ]:
flavio.sm_prediction('<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
wc_np = flavio.WilsonCoefficients()
wc_np.set_initial({'C9_bsmumu': -1.2}, scale=4.2)
flavio.np_prediction('<Rmue>(B+->Kll)', q2min=1, q2max=6, wc_obj=wc_np)

### Impact of NP on uncertatinties

In $R_K$, form factor uncertainties cancel out exactly in the SM, but not in the presence of NP in numerator *or* denominator *only*

In [ ]:
flavio.sm_uncertainty('<Rmue>(B+->Kll)', q2min=1, q2max=6)

In [ ]:
flavio.np_uncertainty('<Rmue>(B+->Kll)', q2min=1, q2max=6, wc_obj=wc_np)

## Interfacing with other codes

- In flavio, NP always enters through Wilson coefficients of local operators
- To investigate dynamical NP *models*, Wilson coefficienst can be computed "by hand" or with a number of public codes
- Some codes already (or will soon) support WCxf, e.g.: `SARAH`/`FlavorKit`, `SPheno`, `FormFlavor`

## Example: importing a WCxf file

In [ ]:
import wcxf


with open('files/wcxf-flavio-example.yml', 'r') as f:
    wc_w = wcxf.WC.load(f)
    
wc_f = flavio.WilsonCoefficients()
wc_f.set_initial_wcxf(wc_w)
wc_f

### Pre-WCxf interface to `SARAH`/`FlavorKit`, `SPheno`

- Based on [FLHA]()
- as of flavio v0.27, still recommended as WCxf interface is not tested sufficiently

Example output file:

In [ ]:
! cat 'SPheno.spc.MSSM'

In [ ]:
import flavio
from flavio.io import flha

my_wc = flha.read_wilson('files/SPheno.spc.MSSM')

In [ ]:
my_wc

In [ ]:
1e4 * flavio.sm_prediction('BR(B->Xsgamma)')

In [ ]:
1e4 * flavio.np_prediction('BR(B->Xsgamma)', wc_obj=my_wc)

### Problems:

- add the binned prediction in the presence of new physics to your $P_5'$ plot
- Find points in WET coefficient space that explain the current anomalies in $B$ physics (only looking at central values)

Observable | a.k.a. | SM | Experiment
-------------|---|---------
`<Rmue>(B+->Kll), q2min=1, q2max=6` | $R_K$ | 1.0 | 0.745
`<Rmue>(B+->K*ll), q2min=0.045, q2max=1.1`| $R_{K^*}^\text{low}$ | 0.93 | 0.66
`<Rmue>(B0->K*ll), q2min=1.1, q2max=6`| $R_{K^*}^\text{interm.}$ | 1.0 | 0.69
`Rtaul(B->Dlnu)`| $R_D$ | 0.28 | 0.41
`Rtaul(B->D*lnu)` | $R_{D^*}$ | 0.25 | 0.30

Next: [Fits](4 Fits.ipynb)